***The Anatomy of a Time Series***

1. **Level:** The average value in the series.
2. **Trend:** The long-term increase or decrease in the data.
3. **Seasonality:** Repeating patterns over a fixed period (higher sales every December).
4. **Noise (Residuals):** The random variation that we can't explain.

***Stationarity:*** A series is stationary if its statistical properties (mean, variance) don't change over time. Most models (like ARIMA) struggle with non-stationary data.
If your data has a trend, it is not stationary. We usually fix this using a technique called **differencing** (subtracting the current value from the previous one).

***Data Splitting:*** We use Time Series Cross-Validation, respecting the chronological order of data.

In [ ]:
%pip install -r requirements.txt --no-cache-dir

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

from statsforecast import StatsForecast
from statsforecast.models import Naive, HistoricAverage, WindowAverage, SeasonalNaive

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("daily_sales_french_bakery.csv", parse_dates=["ds"])
df = df.groupby("unique_id").filter(lambda x: len(x) >= 28)
df = df.drop(["unit_price"], axis=1)

df.head()

In [ ]:
plot_series(df=df, ids=["BAGUETTE", "CROISSANT"], palette="viridis")

In [ ]:
plot_series(df=df, ids=["BAGUETTE", "CROISSANT"], max_insample_length=56, palette="viridis")

In [ ]:
horizon = 7

models = [
    Naive(),
    HistoricAverage(),
    WindowAverage(window_size=7),
    SeasonalNaive(season_length=7)
]

sf = StatsForecast(models=models, freq="D")
sf.fit(df=df)
preds = sf.predict(h=horizon)

In [ ]:
preds.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=preds,
    ids=["BAGUETTE", "CROISSANT"],
    max_insample_length=28,
    palette="viridis"
)

In [ ]:
test = df.groupby("unique_id").tail(7)
train = df.drop(test.index).reset_index(drop=True)

In [ ]:
sf.fit(df=train)
preds = sf.predict(h=horizon)
eval_df = pd.merge(test, preds, "left", ["ds", "unique_id"])

In [ ]:
evaluation = evaluate(
    eval_df,
    metrics=[mae]
)

evaluation.head()

In [ ]:
evaluation = evaluation.drop(["unique_id"], axis=1).groupby("metric").mean().reset_index()
evaluation

In [ ]:
methods = evaluation.columns[1:].to_list()
values = evaluation.iloc[0, 1:].to_list()

plt.figure(figsize=(10,6))
bars = plt.bar(methods, values)

for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height()+0.05,
             f"{value:.3f}", ha="center", va="bottom", fontweight="bold")
    
plt.xlabel("Methods")
plt.ylabel("Mean Absolute Error (MAE)")
plt.tight_layout()

plt.show()

In [ ]:
# https://www.youtube.com/watch?v=fxx_E0ojKrc